In [1]:
import pandas as pd
train_labels = pd.read_csv('/data1/hongn/kinetics-dataset/k400/annotations/train.csv')

In [2]:
train_labels.head()

,label,youtube_id,time_start,time_end,split,is_cc
0,abseiling,-3B32lodo2M,59,69,train,0
1,abseiling,-7kbO0v4hag,107,117,train,0
2,abseiling,-bwYZwnwb8E,13,23,train,0
3,abseiling,-Cv3NwxG_8g,87,97,train,0
4,abseiling,-hLv_HL6UhY,151,161,train,0


In [3]:
train_labels2 = pd.read_csv('/data1/hongn/kinetics-dataset/k400/train.csv')
train_labels2.head()

,/data1/hongn/kinetics-dataset/k400/train/8mbN0ubh0q8_000000_000010.mp4,101
0,/data1/hongn/kinetics-dataset/k400/train/9bfc-...,17
1,/data1/hongn/kinetics-dataset/k400/train/VV0KA...,167
2,/data1/hongn/kinetics-dataset/k400/train/sWCnx...,318
3,/data1/hongn/kinetics-dataset/k400/train/Am6c9...,356
4,/data1/hongn/kinetics-dataset/k400/train/pC1hX...,231


In [13]:
query = '/data1/hongn/kinetics-dataset/k400/replacement/replacement_for_corrupted_k400/NNl5SiIQHsA_000009_000019.mp4'
query in listreplace

True

In [ ]:
/data1/hongn/kinetics-dataset/k400/train/iVsscJC94-c_000225_000235.mp4

In [11]:
import glob
listreplace = glob.glob('/data1/hongn/kinetics-dataset/k400/replacement/replacement_for_corrupted_k400/*')
for i in listreplace:

['/data1/hongn/kinetics-dataset/k400/replacement/replacement_for_corrupted_k400/hpHsbuBa9sI_000009_000019.mp4',
 '/data1/hongn/kinetics-dataset/k400/replacement/replacement_for_corrupted_k400/cvydQIiWOXw_000019_000029.mp4',
 '/data1/hongn/kinetics-dataset/k400/replacement/replacement_for_corrupted_k400/6ZkCVUkeCSA_000033_000043.mp4',
 '/data1/hongn/kinetics-dataset/k400/replacement/replacement_for_corrupted_k400/XGuQmobjS38_000010_000020.mp4',
 '/data1/hongn/kinetics-dataset/k400/replacement/replacement_for_corrupted_k400/xnuPhhcDLcQ_000423_000433.mp4',
 '/data1/hongn/kinetics-dataset/k400/replacement/replacement_for_corrupted_k400/DA_-jEoeWvQ_000010_000020.mp4',
 '/data1/hongn/kinetics-dataset/k400/replacement/replacement_for_corrupted_k400/4ee9AXLznJQ_000730_000740.mp4',
 '/data1/hongn/kinetics-dataset/k400/replacement/replacement_for_corrupted_k400/ce2mbp2dtQU_000003_000013.mp4',
 '/data1/hongn/kinetics-dataset/k400/replacement/replacement_for_corrupted_k400/oVWrmexY4kw_000017_00002

In [1]:
import os
os.environ['OPENBLAS_NUM_THREADS'] = '1'

import json
import torch
import numpy as np
import torchaudio
from transformers import AutoModel, Wav2Vec2Processor, VivitModel, VivitImageProcessor, VivitConfig, AutoImageProcessor, TimesformerModel
from tqdm import tqdm
import cv2
import argparse


parser = argparse.ArgumentParser(description="Feature extraction script")
parser.add_argument('--input_dir', type=str, required=True, help='Main data directory')
parser.add_argument('--output_dir', type=str, required=True, help='Output directory for features')
args = parser.parse_args()

def create_phoneme_map():
    """Create a mapping of phonemes to indices."""
    vowels = ['AA', 'AE', 'AH', 'AO', 'AW', 'AY', 'EH', 'ER', 'EY', 'IH', 'IY', 'OW', 'OY', 'UH', 'UW']
    consonants = ['B', 'CH', 'D', 'DH', 'F', 'G', 'HH', 'JH', 'K', 'L', 'M', 'N', 'NG', 'P', 'R', 'S', 'SH', 
                  'T', 'TH', 'V', 'W', 'Y', 'Z', 'ZH']
    phonemes = vowels + consonants
    phoneme_map = {p: i for i, p in enumerate(phonemes)}
    phoneme_map['sil'] = len(phoneme_map)  # Add silence token
    return phoneme_map


def align_phonemes_to_sequence(sequence_len, total_duration, phoneme_json):
    """Align phoneme labels to the feature sequence."""
    time_step_duration = total_duration / sequence_len
    aligned_labels = ['sil'] * sequence_len  # Default to silence

    for entry in phoneme_json:
        phoneme = entry['phoneme']
        start_time = entry['start']
        end_time = entry['end']

        # Map phoneme to sequence indices
        start_idx = int(start_time / time_step_duration)
        end_idx = int(end_time / time_step_duration)

        # Assign phoneme to all corresponding time steps
        for idx in range(start_idx, min(end_idx + 1, sequence_len)):
            aligned_labels[idx] = phoneme
        
    return aligned_labels


def generate_labels(audio_len, video_len, total_duration, phoneme_json):
    """Generate phoneme labels for audio, video, and unified sequences."""
    audio_labels = align_phonemes_to_sequence(audio_len, total_duration, phoneme_json)
    video_labels = align_phonemes_to_sequence(video_len, total_duration, phoneme_json)
    unified_labels = align_phonemes_to_sequence(max(audio_len, video_len), total_duration, phoneme_json)
    return audio_labels, video_labels, unified_labels


def extract_audio_features(processor, model, audio_path, device):
    """Extract audio features using Wav2Vec2."""
    waveform, sr = torchaudio.load(audio_path)

    # Resample if necessary
    if sr != 16000:
        waveform = torchaudio.transforms.Resample(sr, 16000)(waveform)

    inputs = processor(
        waveform.squeeze().numpy(),
        sampling_rate=16000,
        return_tensors="pt",
        padding=True
    )
    input_values = inputs["input_values"].to(device)  # Shape: [1, sequence_length]

    with torch.no_grad():
        audio_features = model(input_values).hidden_states[8]  # Shape: [1, seq_len, hidden_size]
    
    print(audio_features.shape)
    return audio_features  # Remove batch dimension


def extract_video_features(image_processor, model, video_path, device, batch_size=32):

    cap = cv2.VideoCapture(video_path)
    frames = []

    # Read all frames from the video
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame = cv2.resize(frame, (64, 64))
        frames.append(frame)

    cap.release()

    frames = np.array(frames, dtype=np.float32)  # Ensure float32 type
    #frames * 255.0  # Model expects ranges [0, 255]

    # Prepare for batch processing
    num_frames = len(frames)
    #print(num_frames)
    features = []

    for i in range(0, num_frames, batch_size):
        batch_frames = frames[i:i + batch_size]  # Get a batch of up to `batch_size` frames

        # If fewer than batch_size frames, pad with black frames (zeros)
        if len(batch_frames) < batch_size:
            pad_frames = batch_size - len(batch_frames)
            padding = np.zeros((pad_frames, *batch_frames[0].shape), dtype=np.float32)
            batch_frames = np.concatenate([batch_frames, padding], axis=0)

        
        video_inputs = image_processor(
            list(batch_frames),  
            return_tensors="pt"
        ).to(device)

       
        with torch.no_grad():
            video_outputs = model(pixel_values=video_inputs.pixel_values)
            batch_features = video_outputs.hidden_states[-1]  # Shape: [B, seq_len, feature_dim]
            features.append(batch_features)

    video_features = torch.cat(features, dim=0)
    
    video_features = video_features.view(-1, video_features.size(-1))
    video_features = torch.unsqueeze(video_features, 0)
    print(f"video features shape: {video_features.shape}")
  

    return video_features


def process_file(filename, args, processor, wavlm_model, image_processor, vivit_model, phoneme_map):
    """Process a single file and extract features."""
    device = next(wavlm_model.parameters()).device

    # Paths
    base_path = os.path.splitext(filename)[0]
    video_path = os.path.join(args.input_dir, 'video', f'{filename}.avi')
    audio_path = os.path.join(args.input_dir, 'audio', f'{filename}.wav')
   # phoneme_path = os.path.join(args.input_dir, 'phonemes', f'{filename}.json')

    # Load phoneme data
    #with open(phoneme_path, 'r') as f:
     #   phoneme_json = json.load(f)

    # Extract features
    audio_features = extract_audio_features(processor, wavlm_model, audio_path, device)
    if torch.any(torch.isnan(audio_features)) or torch.any(torch.isinf(audio_features)):
                print("NaN or Inf found in audio features.")
    video_features = extract_video_features(image_processor, vivit_model, video_path, device)
    if torch.any(torch.isnan(video_features)) or torch.any(torch.isinf(video_features)):
                print("NaN or Inf found in video features.")

    # Generate labels
   # total_duration = phoneme_json[-1]['end']  # End time of the last phoneme
    #unified_labels = align_phonemes_to_sequence(512, total_duration, phoneme_json
    #)

    # Save features and labels
    chunk_name = f"{base_path}"
    torch.save(audio_features, os.path.join(args.output_dir, 'wavlm_feats', f'{chunk_name}.pt'))
    torch.save(video_features, os.path.join(args.output_dir, 'tf_feats', f'{chunk_name}.pt'))
   # torch.save(unified_labels, os.path.join(args.output_dir, 'phoneme_labs', f'{chunk_name}.pt'))


def main():

    device = torch.device("cuda:0" if torch.cuda.is_available() else 'cpu')

    # Create output directories
    os.makedirs(os.path.join(args.output_dir, 'wavlm_feats'), exist_ok=True)
    os.makedirs(os.path.join(args.output_dir, 'tf_feats'), exist_ok=True)
    #os.makedirs(os.path.join(args.output_dir, 'phoneme_labs'), exist_ok=True)

    # Load models and processors
    processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
    wavlm_model = AutoModel.from_pretrained("microsoft/wavlm-base", 
                                             output_hidden_states=True).to(device)
    
    image_processor = AutoImageProcessor.from_pretrained("MCG-NJU/videomae-base")
    model = TimesformerModel.from_pretrained("facebook/timesformer-base-finetuned-k400", output_hidden_states=True, ignore_mismatched_sizes=True
            ).to(device)

    #image_processor = VivitImageProcessor.from_pretrained("google/vivit-b-16x2-kinetics400")
    #vivit_config = VivitConfig.from_pretrained("google/vivit-b-16x2-kinetics400",tubelet_size=[16, 16, 16], output_hidden_states=True, ignore_mismatched_sizes=True)
    #vivit_model = VivitModel.from_pretrained(
     #   "google/vivit-b-16x2-kinetics400", output_hidden_states=True, ignore_mismatched_sizes=True
    #).to(device)

    phoneme_map = create_phoneme_map()

    # Process files
    video_files = [f.split('.')[0] for f in os.listdir(os.path.join(args.input_dir, 'video')) if f.endswith('.avi')]
    for filename in tqdm(video_files, desc="Processing files"):
        process_file(filename, args, processor, wavlm_model, image_processor, model, phoneme_map)


if __name__ == "__main__":
    main()

OSError: libtorch_cuda_cpp.so: cannot open shared object file: No such file or directory

In [274]:
from transformers import AutoModel, Wav2Vec2Processor, VivitModel, VivitImageProcessor, VivitConfig, AutoImageProcessor, TimesformerModel
from tqdm import tqdm
import cv2
import argparse
import time

cap = cv2.VideoCapture("/data2/hongn/TimeSformer/usc_s1_2_0.avi")
frames = []
image_processor = AutoImageProcessor.from_pretrained("MCG-NJU/videomae-base")
# Read all frames from the video
while True:
    ret, frame = cap.read()
    fps = cap.get(cv2.CAP_PROP_FPS)      # OpenCV v2.x used "CV_CAP_PROP_FPS"
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    duration = frame_count/fps
    if not ret:
        break
    
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame = cv2.resize(frame, (64, 64))
    frames.append(frame)
cap.release()
frames = np.array(frames, dtype=np.float32)  # Ensure float32 type
#frames * 255.0  # Model expects ranges [0, 255]
# Prepare for batch processing
num_frames = len(frames)
print("num_frames original" , num_frames)


num_frames original 2907


In [281]:
duration*100 - 2907

29.06706393293598

In [280]:
duration

29.36067063932936

In [257]:
HOP_LENGTH = 320
NUM_FRAMES = 4
FRAME_RATE = 100
TIME_STEPS = 0.02 # seconds
FRAME_STEPS = int(TIME_STEPS * FRAME_RATE) # Frame per seconds, should be integer

## UPSAMPLE by DUPPLICATE from 99 FPS to 100 FPS
y, sr = librosa.load("/data2/hongn/TimeSformer/usc_s1_2_0.wav", sr = 16000)
rtmri, _ = librosa.effects.trim(y)

missing_frames_from_framerate = int(len(rtmri) / HOP_LENGTH * FRAME_STEPS) - num_frames
count = 0
temp = list(frames)
for i in list(range(0, num_frames, int(num_frames / missing_frames_from_framerate)))[:missing_frames_from_framerate]:
    temp.insert(i+count, frames[i+count])
    count += 1
frames = temp
print("num_frames inpterpolate framerate 100:" , len(frames))

num_frames inpterpolate framerate 100: 2936


In [268]:
## Get all videos
video_all = [] # Dimension (B, T, C, W, H)
for i in range(NUM_FRAMES): # Append NUM_FRAMES to the end of video
    frames.append(frames[-1])

frame_tensor = image_processor(list(frames), return_tensors="pt").pixel_values[0]
for i in range(0, len(frames) - NUM_FRAMES, FRAME_STEPS):
    video_all.append(frame_tensor[i:i + NUM_FRAMES])

video_all = torch.stack(video_all)

In [271]:
# Put videos in small batches for computational efficient
BATCH_SIZE = 8
features = []
device = 'cuda:2'
model = TimesformerModel.from_pretrained("facebook/timesformer-base-finetuned-k400", output_hidden_states=True, ignore_mismatched_sizes=True
        ).to(device)

for i in range(0, len(video_all), BATCH_SIZE):
        video_batch = video_all[i:i + BATCH_SIZE]  # Get a batch of up to `batch_size` frames

    # If fewer than batch_size frames, pad with black frames (zeros)
        if len(video_batch) < BATCH_SIZE:
                pad_frames = BATCH_SIZE - len(video_batch)
                padding = torch.zeros((pad_frames, 8, 3, 224, 224), dtype=torch.float32)
                video_batch = torch.concat([video_batch, padding], axis=1)

        with torch.no_grad():
                video_outputs = model(pixel_values=video_batch.to(device))
                batch_features = video_outputs.last_hidden_state[:,0,:]  # Shape: [B, seq_len, feature_dim]
                features.append(batch_features)

features = torch.concat(features , axis=0)

In [273]:
features.shape

torch.Size([1472, 768])

In [245]:
features.shape

AttributeError: 'list' object has no attribute 'shape'

In [168]:
torch.concat([a, padding], axis=0).shape

torch.Size([8, 4, 3, 224, 224])

In [135]:
device = 'cuda:1'
model = TimesformerModel.from_pretrained("facebook/timesformer-base-finetuned-k400", output_hidden_states=True, ignore_mismatched_sizes=True
        ).to(device)

In [ ]:
video_outputs = model(pixel_values=video_inputs)

In [137]:
with torch.no_grad():
    video_outputs = model(pixel_values=video_batch.to(device))
    batch_features = video_outputs.last_hidden_state[:,0,:]  # Shape: [B, seq_len, feature_dim]
    features.append(batch_features)

OutOfMemoryError: CUDA out of memory. Tried to allocate 3.29 GiB. GPU 1 has a total capacity of 47.50 GiB of which 1.01 GiB is free. Including non-PyTorch memory, this process has 46.48 GiB memory in use. Of the allocated memory 38.23 GiB is allocated by PyTorch, and 7.77 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [111]:
batch_frames.shape

(8, 64, 64, 3)

In [113]:
batch_size = 8
device = 'cuda:1'
model = TimesformerModel.from_pretrained("facebook/timesformer-base-finetuned-k400", output_hidden_states=True, ignore_mismatched_sizes=True
        ).to(device)

for i in range(0, num_frames, batch_size):
    batch_frames = frames[i:i + batch_size]  # Get a batch of up to `batch_size` frames

    # If fewer than batch_size frames, pad with black frames (zeros)
    if len(batch_frames) < batch_size:
        pad_frames = batch_size - len(batch_frames)
        padding = np.zeros((pad_frames, *batch_frames[0].shape), dtype=np.float32)
        batch_frames = np.concatenate([batch_frames, padding], axis=0)
    
    video_inputs = image_processor(
        list(batch_frames),  
        return_tensors="pt"
    ).to(device)
    
    with torch.no_grad():
        video_outputs = model(pixel_values=video_inputs.pixel_values)
        batch_features = video_outputs.last_hidden_state[:,0,:]  # Shape: [B, seq_len, feature_dim]
        features.append(batch_features)

video_features = torch.cat(features, dim=0)

video_features = video_features.view(-1, video_features.size(-1))
video_features = torch.unsqueeze(video_features, 0)
print(f"video features shape: {video_features.shape}")

video features shape: torch.Size([1, 364, 768])


In [58]:
import librosa

In [59]:
librosa.feature.melspectrogram

<function librosa.feature.spectral.melspectrogram(*, y: Optional[numpy.ndarray] = None, sr: float = 22050, S: Optional[numpy.ndarray] = None, n_fft: int = 2048, hop_length: int = 512, win_length: Optional[int] = None, window: Union[str, Tuple[Any, ...], float, Callable[[int], numpy.ndarray], numpy._typing._array_like._SupportsArray[numpy.dtype[Any]], numpy._typing._nested_sequence._NestedSequence[numpy._typing._array_like._SupportsArray[numpy.dtype[Any]]], bool, int, complex, bytes, numpy._typing._nested_sequence._NestedSequence[Union[bool, int, float, complex, str, bytes]]] = 'hann', center: bool = True, pad_mode: Union[Literal['constant', 'edge', 'linear_ramp', 'reflect', 'symmetric', 'empty'], Callable[..., Any]] = 'constant', power: float = 2.0, **kwargs: Any) -> numpy.ndarray>

In [82]:
y, sr = librosa.load("/data2/hongn/TimeSformer/usc_s1_2_0.wav", sr = 16000)
rtmri, _ = librosa.effects.trim(y)

S = librosa.feature.melspectrogram(y=rtmri, sr=16000, n_fft=512, hop_length=320, n_mels=128)


In [81]:
0.02 * 128 / 0.008

320.0

In [77]:
rtmri.shape

(469800,)

In [ ]:
len(rtmri) / hop_length * 2

In [272]:
469800 / 320 * 2 / 2

1468.125

In [80]:
1/16000*500

0.03125

In [86]:
S.shape

(128, 1469)

In [85]:
647444/5059

127.9786519074916

In [84]:
audio_transform = librosa.feature.melspectrogram(sample_rate=16000,
    n_fft=512,
    hop_length=128,
    n_mels=128,
    center=True,
    pad_mode='constant'
    )

ParameterError: Input signal must be provided to compute a spectrogram